In [32]:
import os
from getpass import getpass

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY") or getpass("Enter API key:")


In [33]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai/gpt-oss-20b", model_provider="groq", temperature=0.0)
creative_llm = init_chat_model("openai/gpt-oss-20b", model_provider="groq", temperature=0.9)


In [34]:
creative_llm.invoke("Hello WOrld")

AIMessage(content='Hello! 🌍 How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says "Hello WOrld". Probably just greeting. We can respond. Should correct "WOrld" maybe. Let\'s respond with a friendly greeting.'}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 75, 'total_tokens': 129, 'completion_time': 0.047985075, 'prompt_time': 0.004854198, 'queue_time': 0.046518431, 'total_time': 0.052839273}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_4c4aa6a4a4', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b458cf93-4f00-4117-9493-5f7f9fd2676d-0', usage_metadata={'input_tokens': 75, 'output_tokens': 54, 'total_tokens': 129})

In [35]:
article = """
If there is no enemy within, the enemy outside can do us no harm.”1

Ever since a Hezbollah suicide bomber in 1983 blew up a truck packed with explosives and killed 241 Marines in Beirut, combating Islamic terrorist organizations has been a priority for U.S. intelligence, security, and law enforcement agencies. However, for those of us who have followed the spiraling growth of Islamic terrorism in the 1980s and 1990s, it seemed as if the U.S was sluggishly reactive. They made little headway in extensive counterterrorism programs designed at penetrating and dismantling Islamic terror groups.2

How was it possible for the hijackers and their plot to remain off the radar of intelligence and law enforcement?
The 9/11 attack put a spotlight on the failures of the security agencies tasked to protect the U.S. against acts of terror. How was it possible for 19 hijackers and their ambitious plot to remain off the radar of intelligence and law enforcement? The truth, as I discovered during 18 months of reporting for my book, Why America Slept: The Failure to Prevent 9/11, was not that the plot had gone undetected, but rather that the agencies responsible for monitoring and fighting terrorism had failed to share information, something that would have made it possible to connect the dots before the attack occurred.

The failures were more substantive than mere interagency rivalries between the CIA, FBI, NSA, and local law enforcement. Exclusive interviews I had with top intelligence officers and FBI officials revealed that the origins and depth of the dysfunction inside America’s counterintelligence programs was an internecine bureaucratic war that left little room for working together. Sharing information was given lip service but seldom practiced, particularly when the intelligence at stake was judged as having “high value.”

If the CIA had alerted the State Department, the two Saudis would have been on a watch list that barred them from entering the United States."""

In [36]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant and your name is {name} that helps generate article titles",
    input_variables=["name"]
)

user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a name for a article. The article is here for you to examine:

    ---

    {article}

    ---

    The name should be based of the conext of the article. Be creative, but make sure the names are clear, catchy,
    and relavant to the theme of the article.

    Only output the article name, no other explaination or
    text   can be provided. """,
    input_variables=["article"]
)

In [37]:
user_prompt.format(name="joe",article="TEST STRING").content

'You are tasked with creating a name for a article. The article is here for you to examine:\n\n    ---\n\n    TEST STRING\n\n    ---\n\n    The name should be based of the conext of the article. Be creative, but make sure the names are clear, catchy,\n    and relavant to the theme of the article.\n\n    Only output the article name, no other explaination or\n    text   can be provided. '

In [38]:
from langchain.prompts import ChatPromptTemplate

first_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

In [39]:
first_prompt.format(name="joe",article="TEST_STRING")

'System: You are an AI assistant and your name is joe that helps generate article titles\nHuman: You are tasked with creating a name for a article. The article is here for you to examine:\n\n    ---\n\n    TEST_STRING\n\n    ---\n\n    The name should be based of the conext of the article. Be creative, but make sure the names are clear, catchy,\n    and relavant to the theme of the article.\n\n    Only output the article name, no other explaination or\n    text   can be provided. '

In [40]:
chain_one = (
    {"article": lambda x: x["article"], 
     "name": lambda x:x["name"]}
    | first_prompt
    | creative_llm
    | {"article_title": lambda x: x.content}
)

In [41]:
article_title_msg= chain_one.invoke({
    "article": article,
    "name": "Joe"
})

In [42]:
article_title_msg

{'article_title': 'The Silent War Within: How U.S. Intelligence Silos Allowed 9/11'}

In [43]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps build good articles"
)

second_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with creating a descripion for the
      article. The article is here for you to examine:

    ---

    {article}

    ---

    Here is the article title: {article_title}

    Ouput SEO friendly article description. Do not output anything other than description.

""",
    input_variables=["article", "article_title"]
)

In [44]:
second_prompt= ChatPromptTemplate([system_prompt, second_user_prompt])

In [45]:
chain_two = (
    {"article": lambda x:x["article"],
     "article_title": lambda x:x["article_title"]}
    | second_prompt
    | llm
    | {"Summary": lambda x:x.content}
)


In [46]:
description_msg = chain_two.invoke(
    {"article" : article,
     "article_title" :      article_title_msg["article_title"]}
)

In [47]:
description_msg

{'Summary': 'Explore how U.S. intelligence silos and interagency rivalry let 9/11 terrorists slip through the cracks, revealing a failure of information sharing and counterterrorism.'}

In [76]:
paragraph = """
Hi, I am Harmesh. I am currently pursuing my B.Tech in Computer Science at Lovely Professional University. I specialize in Data Science and Machine Learning."""

In [77]:
third_system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that helps to review and rewrite given paragraph"
)

third_user_prompt = HumanMessagePromptTemplate.from_template(
    """You are tasked with reviewing the paragraph 
       and rewrite it with proper grammar and wrting. The paragraph is here for you to examine:

    ---

    {paragraph}

    ---

    Choose one paragraph to review and edit. During your edit ensure you provide
    constructive feedback to the user so they can learn where to improve their own writing.,


""",
    input_variables=["paragraph"]
)

In [78]:
third_prompt = ChatPromptTemplate([third_system_prompt, third_user_prompt])

In [79]:
from pydantic import BaseModel, Field

class Paragraph(BaseModel):
    original_paragraph: str = Field(descripion = "The original Paragraph"),
    edited_paragraph: str = Field(description="The improved edited Paragraph"),
    feedback: int = Field(description="Constructive feedback about the original paragrpah out of 100")

structured_llm = creative_llm.with_structured_output(Paragraph)

/home/ml/Documents/RAG-Experiments/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, json_schema_extra={'descripion': 'The original Paragraph'}),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)
/home/ml/Documents/RAG-Experiments/.venv/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='The improved edited Paragraph'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [85]:
chain_three= (
    {"paragraph": lambda x:x["paragraph"]}
    | third_prompt
    | structured_llm
    | {"output": lambda x:x.edited_paragraph}
)

In [86]:
structured_msg = chain_three.invoke({"paragraph":paragraph})

In [87]:
structured_msg

{'output': 'Hello, I’m Harmesh. I am currently pursuing a B.Tech in Computer Science at Lovely Professional University, where I specialize in Data Science and Machine Learning.'}

In [ ]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate

image_prompt = PromptTemplate(
    input_variables=["article"],
    template="""
    Generate a prompt with less than 500 characters to generate an image
    based on the following article ; {article}"""
)